_Authors: Andreia Dourado, Bruno Moraes_

_Adapted from Melissa Graham example notebook.

__Description: This is a first notebook to create a training set of galaxies using a random sample from the Rubin Science Platform DP0.2 data__


In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tables_io
import qp
import sys 
from matplotlib import gridspec

### 1. Reading the data

In [ ]:
data = pd.read_csv("/home/andreia.dourado/data/random_data.csv", usecols=['objectId','ra','dec','redshift','u_Mag','g_Mag','r_Mag','i_Mag','z_Mag','y_Mag'])

In [ ]:
data

### 2. Magnitude uncertainties

The photometric error was adapted from Melissa Grahram's notebook, according to section 3.2.1 of Ivezic et al(2019), given to:

$$ \sigma_{\rm rand}^2 = (0.04 − \gamma)x + \gamma x^2 $$

where $ x = 10^{0.4(m - m_5)}$, $m_5$ is the $5 \sigma$ limiting magnitude, $m$ is the galaxy magnitude and $\gamma$ has the values fixed below for LSST's filters conditions.


In [ ]:
m5 = [26.09, 27.38, 27.53, 26.83, 26.06, 24.86]
gamma = [0.037, 0.038, 0.039, 0.039, 0.04, 0.04]
bands = ['u', 'g', 'r', 'i', 'z', 'y']

In [ ]:
for f, filt in enumerate(bands):
    temp = np.power(10, 0.4*(data[filt+'_Mag'] - m5[f]))
    data['mag_err_'+filt+'_lsst'] = np.sqrt( (0.04-gamma[f])*temp + (gamma[f]*temp**2) )
    del temp
    
    #uncertainty floor
    tx = np.where(data.loc[:, 'mag_err_'+filt+'_lsst'] < 0.005)[0]
    data.loc[tx, 'mag_err_'+filt+'_lsst'] = 0.005
    del tx
    
    #galaxy with error>0.2 not to be used
    tx = np.where(data.loc[:, 'mag_err_'+filt+'_lsst'] > 0.2)[0]
    data.loc[tx, 'mag_err_'+filt+'_lsst'] = float('NaN')
    del tx
  

In [ ]:
data

### 3. Generate observed magnitudes

Getting a normal distribuition with a standart deviation equal to the uncertainty, it was added a random value from this distribuition in the true magnitude for each galaxy. 

In [ ]:
for f, filt in enumerate(bands):
    data['mag_'+filt+'_lsst'] = data[filt+'_Mag'] + \
                                       (np.random.normal(size=len(data)) * \
                                        data['mag_err_'+filt+'_lsst'])
    
    tx = np.where(np.isnan(data.loc[:, 'mag_err_'+filt+'_lsst']))[0]
    data.loc[tx, 'mag_'+filt+'_lsst'] = float('NaN')
    del tx

In [ ]:
data

### 4. Saving the file

In [ ]:
data_training_set=data[['objectId','ra','dec','redshift','mag_u_lsst','mag_g_lsst','mag_r_lsst','mag_i_lsst','mag_z_lsst','mag_y_lsst','mag_err_u_lsst','mag_err_g_lsst','mag_err_r_lsst','mag_err_i_lsst','mag_err_z_lsst','mag_err_y_lsst']]

In [ ]:
data_training_set

In [ ]:
data_training_set.to_csv('traning_set.csv')

### 2. Analysing the training set

#### 2.1 Funtions

In [ ]:
def plot_errors(catalog):
    bands = ['u', 'g', 'r', 'i', 'z', 'y']
    j=1
    plt.figure(figsize=[9,13])
    for i, band in enumerate(bands): 
        plt.subplot(3,2,j)
        mag = np.array(catalog[f'mag_{band}_lsst'])
        err = np.array(catalog[f'mag_err_{band}_lsst'])
        sn = 1/(10**(0.4*err)-1)
        plt.hexbin(mag,sn , None, mincnt=1, cmap='inferno', gridsize=[200,100], bins='log')
        cbar = plt.colorbar()
        plt.ylabel("S/R ", fontsize=13)
        plt.xlabel("mag "+band, fontsize=13)
        plt.ylim(0,100)
        plt.xlim(20,30)
        plt.axhline(10, color= 'red')
        plt.grid(True)
        j+=1
        plt.tight_layout()
        plt.savefig('errors_training_set.png', format='png')

In [ ]:
def mag_histogram(catalog, title='DP0.2'):
    bands = ['u','g', 'r', 'i', 'z','y']
    colors = ['blue', 'green', 'orange','red','purple','gray']
    plt.figure(figsize=(9,13))
    bins = np.linspace(9, 37, 57)
    j=1
    for i, (band, color) in enumerate(zip(bands,colors)):
        plt.subplot(3,2,j)
        plt.hist(catalog[f'mag_{band}_lsst'], histtype='stepfilled', bins=bins, label=f'{band} band', alpha = 0.5,
                 edgecolor = "black", color = color)
        #plt.xlim(16,27)
        plt.yscale('log')
        plt.xlabel('mag',fontsize=13)
        plt.ylabel('counts',fontsize=13)
        plt.legend(loc=2)
        plt.grid(True)
        j+=1
    plt.suptitle(title)
    plt.savefig('mag_histogram_training_set.png', format='png')
    plt.show()

In [ ]:
def redshift_hist(catalog):
    plt.hist(catalog['redshift'], bins=np.linspace(0,3,200))
    plt.savefig('redshift_training_set.png', format='png')
    plt.show()

In [ ]:
def mag_color(catalog):
    bands = ['u', 'g', 'r', 'i', 'z','y']
    mag_diff = {}
    plt.figure(figsize=(9,13))
    i=1
    for band,_band in zip(bands, bands[1::]):
        plt.subplot(3,2,i)
        i+=1
        mag_diff_v = catalog[f'mag_{band}_lsst']-catalog[f'mag_{_band}_lsst']
        mag_v = catalog[f'mag_{band}_lsst']
        plt.hexbin(mag_v, mag_diff_v, None, mincnt=1, cmap='Reds', gridsize=[400,200], bins='log')
        plt.xlabel("mag "+band,fontsize=13)
        plt.ylabel(f"{band}-{_band}",fontsize=13)
        #plt.legend()
        plt.xlim(16,32)
        plt.ylim(-2,5)
        plt.grid(True)
        plt.tight_layout()
        plt.savefig('magColor_training_set.png', format='png')
    plt.show()

In [ ]:
def color_color(catalog):
    bands = ['u', 'g', 'r', 'i', 'z','y']
    i=1
    plt.figure(figsize=(12,12))
    for index in range(len(bands)-2):
        plt.subplot(3,2,i)
        i+=1
        color = catalog[f'mag_{bands[index+1]}_lsst']
        next_color = catalog[f'mag_{bands[index+2]}_lsst']
        past_color = catalog[f'mag_{bands[index]}_lsst']
        plt.hexbin(past_color-color,color-next_color, None, mincnt=1, cmap='turbo', gridsize=[400,200], bins='log')
        plt.xlabel(f'{bands[index+1]}-{bands[index+2]}',fontsize=13)
        plt.ylabel(f'{bands[index]}-{bands[index+1]}',fontsize=13)
        cbar = plt.colorbar()
        plt.xlim(-1,3)
        plt.ylim(-1,3)
        plt.savefig('colorColor_training_set.png', format='png')
    plt.show()

In [ ]:
def color_color_red(catalog,xlim=[-1,3],ylim=[-1,3]):
    bands = ['u', 'g', 'r', 'i', 'z','y']
    i=1
    plt.figure(figsize=(12,12))
    for index in range(len(bands)-2):
        plt.subplot(3,2,i)
        i+=1
        color = catalog[f'mag_{bands[index+1]}_lsst']
        next_color = catalog[f'mag_{bands[index+2]}_lsst']
        past_color = catalog[f'mag_{bands[index]}_lsst']
        plt.hexbin(past_color-color,color-next_color, C=catalog['redshift'], mincnt=1, cmap='turbo', gridsize=[400,200])
        plt.xlabel(f'{bands[index+1]}-{bands[index+2]}',fontsize=13)
        plt.ylabel(f'{bands[index]}-{bands[index+1]}',fontsize=13)
        cbar = plt.colorbar(label='redshift')
        plt.xlim(xlim[0],xlim[1])
        plt.ylim(ylim[0],ylim[1])
        plt.savefig('colorColorRed_training_set.png', format='png')
    plt.show()

In [ ]:
def spatial_distribution(catalog):
    plt.scatter(catalog['ra'],catalog['dec'], c=catalog['redshift'],cmap='turbo',s=1)
    plt.xlabel('RA [deg]')
    plt.ylabel('Dec [deg]')
    plt.colorbar()
    plt.savefig('spatial_distribution.png', format='png')

#### 2.2 Plots

Spatial distribution

In [ ]:
spatial_distribution(data_training_set)

Redshift distribution

In [ ]:
redshift_hist(data_training_set)

Errors

In [ ]:
plot_errors(data_training_set)

Magnitude distribution

In [ ]:
mag_histogram(data_training_set)

Mag-color

In [ ]:
mag_color(data_training_set)

Color-color

In [ ]:
color_color(data_training_set)

In [ ]:
color_color_red(data_training_set)